<span style="font-weight: bold; font-size:30px; color:#2196F3">11-2 데이터 변환</span>

11-1에서는 집계 메서드의 활용법을 알아보았다. 이번에는 데이터 변환 메서드에 대해 알아보겠다. 데이터 변환 메서드는 데이터와 메서드를 일대일로 대응시켜 계산하기 때문에 데이터의 양이 줄어들지 않는다. 말 그대로 데이터를 변환하는 데 사용한다.

<span style="font-weight: bold; font-size:20px; color:#2196F3">표준점수 계산하기</span><br>통계 분야에서는 데이터의 평균과 표준편차의 차이를 표준점수라고 부른다. 표준점수를 구하면 변환한 데이터의 평균값이 0이 되고 표준편차는 1이 된다. 그러면 데이터가 표준화되어 서로 다른 데이터를 쉽게 비교할 수 있게 된다. 표준점수는 통계에서 자주 사용하는 지표이다.

### 표준점수 계산하기
#### 다음은 표준점수를 계산하는 함수이다.

In [1]:
def my_zscore(x):
    return (x - x.mean())/ x.std()

#### 다음은 각 연도별 lifeExp 열의 표준점수를 계산한 것이다. my_zscore 함수를 적용하기 위해 transform 메서드를 사용했다.

In [3]:
import pandas as pd
df = pd.read_csv('doit_pandas_data/data/gapminder.tsv', sep = '\t')

transform_z = df.groupby('year').lifeExp.transform(my_zscore)
print(transform_z.head())

0   -1.656854
1   -1.731249
2   -1.786543
3   -1.848157
4   -1.894173
Name: lifeExp, dtype: float64


#### my_zscore 함수는 데이터를 표준화할 뿐 집계는 하지 않는다. 즉, 데이터의 양이 줄어들지 않는다. 다음은 원본 데이터프레임(df)의 데이터 크기와 변환한 데이터프레임의 데이터 크기를 비교한 것이다.

In [4]:
print(df.shape)

(1704, 6)


In [5]:
print(transform_z.shape)

(1704,)


<span style="font-weight: bold; font-size:20px; color:#2196F3">누락값을 평균값으로 처리하기</span><br>06장에서는 누락값을 처리하는 다양한 방법에 대해 살펴봤다. 하지만 가끔은 누락값을 평균값으로 처리하는 것이 더 좋을 때가 있다. 이번에는 누락값을 평균값으로 처리하는 방법을 알아보겠다.

### 누락값을 평균값으로 처리하기
#### 다음은 seaborn 라이브러리의 tips 데이터 집합에서 10개의 행 데이터만 가져온 다음 total_bill 열의 값 4개를 임의로 선택하여 누락값으로 바꾼 것이다.

In [6]:
import seaborn as sns
import numpy as np

np.random.seed(42)
tips_10 = sns.load_dataset('tips').sample(10)
tips_10.loc[np.random.permutation(tips_10.index)[:4], 'total_bill'] = np.NaN
print(tips_10)

     total_bill   tip     sex smoker   day    time  size
24        19.82  3.18    Male     No   Sat  Dinner     2
6          8.77  2.00    Male     No   Sun  Dinner     2
153         NaN  2.00    Male     No   Sun  Dinner     4
211         NaN  5.16    Male    Yes   Sat  Dinner     4
198         NaN  2.00  Female    Yes  Thur   Lunch     2
176         NaN  2.00    Male    Yes   Sun  Dinner     2
192       28.44  2.56    Male    Yes  Thur   Lunch     2
124       12.48  2.52  Female     No  Thur   Lunch     2
9         14.78  3.23    Male     No   Sun  Dinner     2
101       15.38  3.00  Female    Yes   Fri  Dinner     2


#### 그런데 total_bill 열의 누락값을 단순히 total_bill 열의 평균값으로 채우면 안된다. 현재 tips_10의 데이터는 여성보다 남성이 더 많다. 즉, 여성과 남성을 구분하여 total_bill 열의 평균값을 구하지 않으면 여성 데이터가 남성 데이터의 영향을 많이 받아 여성의 데이터가 훼손될 수 있다. 다음은 성별로 그룹화한 다음 각 열의 데이터 수를 구한 것이다. total_bill 열을 살펴보면 남성의 누락값은 3개, 여성의 누락값은 1개라는 것을 알 수 있다.

In [7]:
count_sex = tips_10.groupby('sex').count()
print(count_sex)

        total_bill  tip  smoker  day  time  size
sex                                             
Male             4    7       7    7     7     7
Female           2    3       3    3     3     3


#### 다음은 성별을 구분하여 total_bill 열의 데이터를 받아 평균을 구하는 함수이다.

In [8]:
def fill_na_mean(x):
    avg = x.mean()
    return x.fillna(avg)

#### 다음은 성별을 구분한 total_bill 열의 데이터를 fill_na_mean 함수에 전달하여 평균값을 구한 다음 tips_10에 새로운 열로 추가한 것이다. 남성과 여성의 누락값을 고려하여 계산한 평균값으로 잘 채워져 있는 것을 알 수 있다.

In [9]:
total_bill_group_mean = tips_10.groupby('sex').total_bill.transform(fill_na_mean)
tips_10['fill_total_bill'] = total_bill_group_mean
print(tips_10)

     total_bill   tip     sex smoker   day    time  size  fill_total_bill
24        19.82  3.18    Male     No   Sat  Dinner     2          19.8200
6          8.77  2.00    Male     No   Sun  Dinner     2           8.7700
153         NaN  2.00    Male     No   Sun  Dinner     4          17.9525
211         NaN  5.16    Male    Yes   Sat  Dinner     4          17.9525
198         NaN  2.00  Female    Yes  Thur   Lunch     2          13.9300
176         NaN  2.00    Male    Yes   Sun  Dinner     2          17.9525
192       28.44  2.56    Male    Yes  Thur   Lunch     2          28.4400
124       12.48  2.52  Female     No  Thur   Lunch     2          12.4800
9         14.78  3.23    Male     No   Sun  Dinner     2          14.7800
101       15.38  3.00  Female    Yes   Fri  Dinner     2          15.3800


<span style="font-weight: bold; font-size:20px; color:Gray">출처 : Do it! 데이터 분석을 위한 판다스</span>